In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import json, argparse, os
from pathlib import Path
import fasttext

In [2]:
label2category = {0:'impact', 1:'black voices', 2:'entertainment', 3:'queer voices', 4:'religion', 5:'parenting', 6:'comedy', 7:'business', 
8:'travel', 9:'arts & culture', 10:'education', 11:'parents', 12:'crime', 13:'money', 14:'worldpost', 15:'food & drink', 16:'sports', 
17:'good news', 18:'culture & arts', 19:'the worldpost', 20:'environment', 21:'politics', 22:'science', 23:'weird news', 24:'arts', 
25:'healthy living', 26:'world news', 27:'media', 28:'latino voices', 29:'college', 30:'divorce', 31:'green', 32:'style', 33:'tech', 
34:'weddings', 35:'home & living', 36:'wellness', 37:'fifty', 38:'style & beauty', 39:'women', 40:'taste'}

In [3]:
category2label = {'impact':0, 'black voices':1, 'entertainment':2, 'queer voices':3, 'religion':4, 'parenting':5, 'comedy':6, 'business':7, 
'travel':8, 'arts & culture':9, 'education':10, 'parents':11, 'crime':12, 'money':13, 'worldpost':14, 'food & drink':15, 'sports':16, 
'good news':17, 'culture & arts':18, 'the worldpost':19, 'environment':20, 'politics':21, 'science':22, 'weird news':23, 'arts':24, 
'healthy living':25, 'world news':26, 'media':27, 'latino voices':28, 'college':29, 'divorce':30, 'green':31, 'style':32, 'tech':33, 
'weddings':34, 'home & living':35, 'wellness':36, 'fifty':37, 'style & beauty':38, 'women':39, 'taste':40}

In [32]:
data = "dataset.json"
training_data = Path("fasttext_dataset_train.txt")
test_data = Path("fasttext_dataset_test.txt")


In [6]:
#Percentage of data in train and test set
percent_test_data = 0.25


In [7]:
#Preprocessing and Formatting
import re
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
ps = PorterStemmer()

In [8]:

def strip_formatting(string):
    string = string.lower()
    string = re.sub('[^a-zA-Z]', ' ', string)
    string = string.split()
    string = [ps.stem(word) for word in string if not word in set(stopwords.words('english'))]
    string = ' '.join(string)
    
    return(string)
    

In [36]:
inputFile = open(data, 'r', encoding="utf8") #open json file
data1 = json.load(inputFile) #load json content
print(type(data1))
print(data1[5])

<class 'list'>
{'category': 'entertainment', 'id': 6, 'short_description': '"It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor," he said in a statement.', 'headline': "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy"}


In [37]:
inputFile = open(data, 'r', encoding="utf8") #open json file
data1 = json.load(inputFile) #load json content
print(type(data1))
with training_data.open("a") as train_output, \
     test_data.open("a") as test_output:

    for line in data1:
        news_data = line
        
        label = news_data['category']
        news_data['text'] = news_data['headline'] + " " + news_data['short_description']
        text = news_data['text'].replace("\n", " ")
        text = strip_formatting(text)

        fasttext_line = "__label__{} {}".format(label, text)

        if random.random() <= percent_test_data:
            test_output.write(fasttext_line + "\n")
        else:
            train_output.write(fasttext_line + "\n")    

<class 'list'>


In [ ]:
#Model Building
            
classifier = fasttext.load_model('model.bin', encoding = 'utf-8')

#model = fasttext.skipgram('fasttext_dataset_train.txt', 'model')

model = fasttext.cbow('fasttext_dataset_train.txt','model',dim=150,ws=5,epoch=5,neg=10,min_count=1)

classifier = fasttext.supervised(input_file="fasttext_dataset_train.txt",
                                     output='model',
                                     dim = 150,
                                     lr=0.01,
                                     epoch = 50,minn=1,maxn=10)
                                     

result = classifier.test('fasttext_dataset_test.txt')
print ('P@1:', result.precision)
print ('R@1:', result.recall)
print ('Number of examples:', result.nexamples)

texts = ['former disney exec launch social network girl maverick aim connect empow young women girl']
print (classifier.predict(texts, k=10))